# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("./Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
amount_spent_per_customer = orders.groupby("CustomerID").sum()
unique_customer_amount = amount_spent_per_customer.loc[:, "amount_spent"]


In [4]:
orders.describe()
Q75 = np.percentile(unique_customer_amount, 75)
Q95 = np.percentile(unique_customer_amount, 95)
Q75, Q95

VIP_clients = unique_customer_amount[unique_customer_amount > Q95]
Preferred_Clients = unique_customer_amount[(unique_customer_amount > Q75) & (unique_customer_amount < Q95)]

In [5]:
orders['Customer Type'] = np.where(orders['CustomerID'].isin(VIP_clients.index), 'VIP', np.where(orders['CustomerID'].isin(Preferred_Clients.index), 'Preferred', 'Regular'))


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [6]:
# your code here Q2
VIPS_per_country = orders.groupby(["Country","Customer Type"]).agg({'CustomerID':'count'})
VIPS_per_country

CustomerID
Country              Customer Type            
Australia            Preferred             130
                     Regular               157
                     VIP                   898
Austria              Preferred             158
                     Regular               240
...                                        ...
United Arab Emirates Regular                68
United Kingdom       Preferred          137450
                     Regular            132710
                     VIP                 84185
Unspecified          Regular               244

[76 rows x 1 columns]

In [7]:
pivot = pd.pivot_table(orders, values='CustomerID', 
            columns='Customer Type', 
            index='Country', 
            aggfunc=({'CustomerID':'count'}))
pivot

Customer Type,Preferred,Regular,VIP
Country,,,
Australia,130.0,157.0,898.0
Austria,158.0,240.0,NaN
Bahrain,NaN,17.0,NaN
Belgium,1503.0,474.0,54.0
Brazil,NaN,32.0,NaN
Canada,135.0,16.0,NaN
Channel Islands,225.0,159.0,364.0
Cyprus,203.0,163.0,248.0
Czech Republic,NaN,25.0,NaN


In [8]:
pivot['VIP'].sort_values(ascending=False).head(1)

Country
United Kingdom    84185.0
Name: VIP, dtype: float64

In [9]:
pivot['VIP_Preferred'] = pivot['VIP'] + pivot['Preferred']
pivot['VIP_Preferred'].sort_values(ascending=False).head(1)

Country
United Kingdom    221635.0
Name: VIP_Preferred, dtype: float64